In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer the question using the following context and chat history, but refer to the chat history first. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        # MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {   
        "context": retriever,
        "question": RunnablePassthrough(),
        # "history": load_memory
    }
    | prompt 
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question) 
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

In [2]:
invoke_chain("Is Aaronson guilty?")

According to the context provided, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having a memory that contradicts this belief. He has convinced himself that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's narrative.


In [3]:
invoke_chain("What message did he write in the table?")

Winston traced "2+2=5" in the dust on the table.


In [4]:
invoke_chain("Who is Julia?")

Julia is a character in the context provided who has a romantic relationship with Winston. She represents a form of rebellion against the oppressive regime of the Party, and Winston has deep feelings for her. Their relationship is central to the themes of love, rebellion, and the struggle against totalitarianism in the narrative.


In [6]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite having a memory that contradicts this belief. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's narrative."),
  HumanMessage(content='What message did he write in the table?'),
  AIMessage(content='He traced with his finger in the dust on the table: 2+2=5.'),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content='Julia is a character who is loved by the protagonist, Winston. She is someone with whom Winston shares a deep emotional connection, and he expresses a strong desire to protect her, even at the cost of his own safety.')]}